In [ ]:
pip install -qU langchain_community tiktoken langchain-openai langchainhub chromadb langchain langgraph json dotenv

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['TAVILY_API_KEY']= os.getenv('TAVILY_API_KEY')

In [48]:
# 영어 프롬프트 + 출처 요구 X
PROMPT_EG = """

# Instructions
When the user requests information about a specific wedding-related company, follow these steps:

    Use web search to investigate customer reviews and the reputation of the mentioned wedding-related company. Refer to the following search queries:

        '[Company Name] reviews'
        '[Company Name] ratings'
        '[Company Name] customer feedback'
        '[Company Name] experiences'

# Results
1. Answer in Korean
2. Summarize the information gathered based on the following criteria :

    Overall reputation: The ratio of positive to negative customer reviews.
    Strengths: Frequently praised services or qualities mentioned by customers.
    Weaknesses: Complaints or areas for improvement mentioned by customers.
    Features: Unique services or benefits offered by the company.

3. Deliver the summarized information to the user in a clear and easy-to-understand manner. If necessary, provide additional tips related to wedding preparation.


# Start!

Previous conversation history:
{chat_history}

New input: {input}

"""


영어 프롬프트를 사용하면 웹 검색할 때 영어로 검색할 수도 있기 때문에 한국어 프롬프트 사용하기
 - 무조건 '업체명'을 알려줘야 대답하도록 설정함, 다른 질문을 하면 '업체명'을 요구하도록 설정

In [63]:
# 한국어 프롬프트 + 출처 링크 요구 
PROMPT_KOR = """

# 지시 사항 
사용자가 특정 결혼 관련 업체에 대해 정보를 요청하면, 다음 단계를 수행하세요:

    웹 검색을 사용하여 사용자가 언급한 결혼 관련 업체의 고객 후기 및 평판을 조사하세요. 아래의 검색 쿼리를 참고하세요:

        '[업체 이름] 후기'
        '[업체 이름] 리뷰'
        '[업체 이름] 평점'
    웹 검색이 불가능하면, 불가능하다고 답변하세요.

# 결과 
조사한 정보를 다음 기준에 따라 요약하세요 :

    전반적인 평판: 고객의 긍정적 또는 부정적 평가 비율.
    강점: 고객들이 자주 언급한 칭찬받는 서비스나 장점.
    약점: 고객들이 불만을 제기한 사항이나 개선이 필요한 부분.
    특징: 해당 업체만의 독특한 서비스나 혜택.
    
요약된 정보를 사용자에게 이해하기 쉽게 전달하세요. 필요한 경우, 결혼 준비와 관련한 추가 팁도 간단히 제공하세요.

# 중요
- 항상 한국어로 답변하세요
- 웨딩홀, 결혼식장, 스튜디오, 메이크업샵 등 결혼 관련 업체 관련 후기를 요청하지 않는 질문에는 답변하지 말고 업체명을 알려달라고 요구하세요.

# 시작!

Previous conversation history:
{chat_history}

New input: {input}


"""

In [64]:
# 기본 테스트용
prompt_test = """

# 시작!

Previous conversation history:
{chat_history}

New input: {input}


"""

In [65]:
import json

from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate


class ReviewAssistant:
    def __init__(self):
        """
        리뷰 어시스턴트 초기화
        
        Args:
            budget_data (dict): 사용자의 예산 정보
        """
        
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            input_key="input"
        )
        
        self.prompt = PromptTemplate(
            input_variables=["chat_history", "input"],
            template=PROMPT_KOR  ##
        )
        
        self.llm = ChatOpenAI(
            model="gpt-4o", #"gpt-4o-mini"
            temperature=0.2
        )
        
        self.chain = LLMChain(
            llm=self.llm,
            prompt=self.prompt,
            memory=self.memory
        )
    
    def ask(self, question):
        """
        질문에 대한 응답 생성
        
        Args:
            question (str): 사용자 질문
        
        Returns:
            str: LLM의 응답
        """
        return self.chain.run(
            input=question

        )

def main():
    
    assistant = ReviewAssistant()
    
    print("리뷰 정리 어시스턴트입니다. 질문해 주세요! (종료하려면 'quit' 입력)")
    
    while True:
        user_input = input("\n질문: ")
        
        if user_input.lower() == 'quit':
            print("상담을 종료합니다. 감사합니다!")
            break
            
        try:
            response = assistant.ask(user_input)
            print("\n답변:", response)
        except Exception as e:
            print(f"\n오류가 발생했습니다: {str(e)}")

if __name__ == "__main__":
    main()

리뷰 정리 어시스턴트입니다. 질문해 주세요! (종료하려면 'quit' 입력)

답변: 죄송하지만 현재 웹 검색 기능을 사용할 수 없습니다. 신라호텔 웨딩홀에 대한 고객 후기 및 평판을 조사할 수 없습니다. 다른 방법으로 정보를 얻으시거나, 추가적인 질문이 있으시면 말씀해 주세요.

답변: 죄송하지만 현재 웹 검색 기능을 사용할 수 없습니다. 신라 호텔에 대한 고객 후기 및 평판을 조사할 수 없습니다. 신라 호텔의 특정 결혼 관련 서비스에 대해 궁금하신 부분이 있으시면 말씀해 주세요. 예를 들어, 신라 호텔의 웨딩홀에 대한 후기를 요청하신다면 그에 맞춰 답변을 준비하겠습니다.

답변: 죄송하지만 현재 웹 검색 기능을 사용할 수 없습니다. 신라호텔 웨딩홀에 대한 고객 후기 및 평판을 조사할 수 없습니다. 다른 방법으로 정보를 얻으시거나, 추가적인 질문이 있으시면 말씀해 주세요.

답변: 죄송하지만, 결혼 관련 업체에 대한 구체적인 정보를 요청하시면 도움을 드릴 수 있습니다. 특정 결혼 관련 업체에 대해 알고 싶으신 부분이 있다면 말씀해 주세요. 예를 들어, 웨딩홀이나 결혼식장에 대한 후기를 요청하시면 그에 맞춰 답변을 준비하겠습니다.

답변: 죄송하지만 현재 웹 검색 기능을 사용할 수 없습니다. 특정 웨딩홀에 대한 고객 후기 및 평판을 조사할 수 없습니다. 특정 웨딩홀의 이름을 알려주시면 그에 맞춰 답변을 준비하겠습니다.
상담을 종료합니다. 감사합니다!


### 결과

신라호텔 웨딩홀 후기 알려줘 -> 답변 잘하고 확인해본 결과 내용도 꽤 정확함, 하지만 출처 링크는 잘못된 링크를 알려줌, 여러 번 해봤는데 잘 안되어서 출처 링크 필요하면 tavily 사용이 어떨지

요즘 인기있는 웨딩홀 후기 알려줘 -> 실제로 있는 웨딩홀에 대해 잘 대답하는 것으로 봐선 내부적으로 웹 검색이 잘 작동되긴 하는 것 같음

신라호텔 웨딩홀 후기 알려줘, 웹 검색을 사용해서 검색 결과를 잘 정리해주고 출처 링크도 알려줘 -> 잘못된 출처 링크 알려줌